# Coding assignment: traffic flow

Nous allons résoudre l'équation d'évolution du trafic de voitures sur une route. C'est une équation à deux variables: le temps et l'espace. L'énoncé nous forunit les égalités suivantes:

\begin{equation}
    V = V_{max}\left(1-\frac{\rho}{\rho_{max}}\right)
\end{equation}

\begin{equation}
    F=\rho V
\end{equation}

\begin{equation}
    \frac{\partial \rho}{\partial t} + \frac{\partial F}{\partial x} = 0
\end{equation}

où $\rho_{max}$ et $V_{max}$ sont des constantes. En combinant la première équation avec la seconde, on a

\begin{equation}
    F = \rho V_{max}\left(1-\frac{\rho}{\rho_{max}}\right)
\end{equation}

Insérant cette expression dans la troisième équation, on a

\begin{align}
    &\frac{\partial \rho}{\partial t} + V_{max} \left(\frac{\partial \rho}{\partial x} \left(1-\frac{\rho}{\rho_{max}}\right) - \frac{\rho}{\rho_{max}} \frac{\partial \rho}{\partial x}\right)=0\\
    &\Leftrightarrow \frac{\partial \rho}{\partial t} + \frac{\partial \rho}{\partial x} V_{max} \left(1- \frac{2 \rho}{\rho_{max}} \right)=0\\
\end{align}

Discrétisons le problème. Nous allons discrétiser les variables: ce seront des grilles de valeurs équidistantes. Les positions $x$ seront données par une série de valeurs {$x_{i}$}, et les temps seront des valeurs {$t^n$} équidistantes.

Comme demandé dans l'énoncé du problème, nous approximerons la dérivée par rapport au temps par un quotient fini de la dérivée à droite de $\rho$ par rapport au temps (forward-difference), et la dérivée par rapport à $x$ sera remplacée par un quotient de la dérivée à gauche (backward-difference):

\begin{align}
    \frac{\rho_{i}^{n+1}-\rho_{i}^n}{\Delta t} + \frac{\rho_{i}^{n}-\rho_{i-1}^{n}}{\Delta x} V_{max} \left(1- \frac{2 \rho_{i}^{n}}{\rho_{max}} \right)=0\\
\end{align}

où $\rho_{i}^n$ désigne la valeur de $\rho$ au temps $t^n$ à la position $x_{i}$. D'autre part, nous connaissons $\rho$ pour tout $x$ au temps $t=0$, c'est à dire pour $n=0$. Dans l'équation pour n=0, il ne nous manque que $\rho_{i}^{n+1}$, nous pouvons l'isoler et ainsi le calculer. Nous connaîtrons $\rho_{i}^1$ pour tout $i$, et pourrons calculer $\rho_{i}^2$, etc.
Isolons $\rho_{i}^{n+1}$ :

\begin{align}
    \rho_{i}^{n+1}=\rho_{i}^n - \Delta t \frac{\rho_{i}^{n}-\rho_{i-1}^{n}}{\Delta x} V_{max} \left(1- \frac{2 \rho_{i}^{n}}{\rho_{max}} \right)
\end{align}

Implémentons cela en code. Il faut commencer par importer les modules:

In [1]:
import numpy
from matplotlib import pyplot  
%matplotlib inline

Nous allons définir une fonction pour résoudre le problème, qui prend en paramètres le $\rho$ initial ($\rho_{0}$), le $\rho$ à $x=0$ en tout temps ($\rho_{lim}$) et la vitesse maximum des voitures ($V_{max}$). Ce sont les 3 paramètres que l'on devra faire varier.

In [2]:
def spacetime(rho0,rholim,Vmax):
    rho=numpy.ones((nt,nx))
    rho[0,:]=rho0
    rho[:,0]=rholim
    
    for n in range(1,nt):
        rho[n,1:] = rho[n-1,1:]-Vmax*dt/dx*((1-rho[n-1,1:]/rhoMax)*rho[n-1,1:]-(1-rho[n-1,:-1]/rhoMax)*rho[n-1,:-1])
        #rho[n,1:] = rho[n-1,1:]-Vmax*dt/dx*((1-2*rho[n-1,1:]/rhoMax)*(rho[n-1,1:]-rho[n-1,:-1]))
    return rho

La fonction commence par initialiser un tableau pour stocker les valeurs de $\rho$ pour toutes les positions et tous les temps. Elle fixe ensuite les valeurs au temps 0 et les valeurs à la position 0 grâce aux arguments donnés. Une boucle parcourt les différentes valeurs du temps et calcule la valeur de $\rho$ au temps suivant pour tous les $x$. Dans cette boucle, nous utilisons les opérations entre vecteurs (tableau numpy à une dimension) permises pour appliquer l'équation discrétisée ci-dessus à toutes les valeurs de $i$ en une seule ligne.
La fonction renvoie ensuite le tableau des valeurs de $\rho$.

Nous avons donc maintenant toutes les valeurs de $\rho$ à notre disposition. Les questions posées sont sur la vitesse, nous allons donc calculer la vitesse à toutes les temps et en toutes les positions par la formule suivante:

\begin{align}
V = V_{max}\left(1-\frac{\rho}{\rho_{max}}\right)
\end{align}

Pour cela, créons une fonction qui prend en argument une valeur de $\rho$ et renvoie la vitesse:

In [3]:
def V(rhoni,Vmax): #V est une fonction de rho[n,i]
    return Vmax*(1-rhoni/rhoMax)

Au lieu d'utiliser rho comme argument de la fonction, nous avons utilisé rhoni (pour rho[n,i]) pour éviter la confusion avec le tableau des valeurs de ρ (utiliser le même nom n'aurait pas posé de problème puisque les deux variables sont dans des fonctions différentes).
Nous pouvons maintenant utiliser ces fonctions. Commençons par initialiser les paramètres (les unités des grandeurs physiques sont en heures et kilomètres):

In [4]:
L=11 #longueur de la route
rhoMax=250
nx=51 #l'espace est discrétisé en 50 valeurs
dx=L/(nx-1) #l'espace est donc discrétisé par pas dx
dt=.001 #le temps est discrétisé par pas de 0.001 heure
nt=int(6/60/dt)+1 #le temps est donc discrétisé en nt valeurs

# Première série de questions

Pour la première série de questions, les paramètres $V_{max}$, $\rho_{0}$ et $\rho_{lim}$ sont les suivants:

In [5]:
Vmax1=80
rho01 = numpy.ones(nx)*10
rho01[10:20] = 50
rholim1=numpy.ones(nt)*10

Appelons maintenant la fonction pour trouver les valeurs $\rho$:

In [6]:
rho1=spacetime(rho01,rholim1,Vmax1)

Appelons maintenant la fonction pour calculer les valeurs de la vitesse. À nouveau, nous utilisons une facilité de numpy: après avoir créé un tableau pour les vitesses, la commande de calcul de celles-ci se fait en une ligne. Les valeurs des vitesses reçoivent par la fonction V l'image de la valeur du tableau de $\rho$ qui a les mêmes indices.

In [7]:
v1=numpy.ones((nt,nx))
v1[:,:]=V(rho1[:,:],Vmax1)

Il reste à utiliser des fonctions fournies par Python pour trouver le minimum de vitesse au temps 0, la moyenne à 3 minutes et le minimum à 6 minutes. La fonction min renvoie le minimum d'un vecteur, et la fonction sum fait les somme des ééments d'un vecteur.

In [9]:
print("La vitesse minimale à t=0 minute est:", min(v1[0,:])/3.6) 
print("La vitesse moyenne à t=3 minutes est:", sum(v1[int(3/60/dt),:])/nx/3.6)
print("Le vitesse minimale à t=6 minutes est:", min(v1[int(6/60/dt),:])/3.6) 

La vitesse minimale à t=0 minute est: 17.7777777778
La vitesse moyenne à t=3 minutes est: 20.6361661961
Le vitesse minimale à t=6 minutes est: 18.7847168709


# Deuxième série de questions

Il nous reste maintenant à répondre aux questions de la seconde série. On peut réutiliser nos programmes et simplement changer les arguments et les conditions initiales.

In [10]:
Vmax2=136
rho02 = numpy.ones(nx)*20
rho02[10:20] = 50
rholim2=numpy.ones(nt)*20

Tout est prêt pour appeler la fonction spacetime:

In [11]:
rho2=spacetime(rho02,rholim2,Vmax2)

Comme précédemment, calculons les vitesses à l'aide de la fonction V:

In [12]:
v2=numpy.ones((nt,nx))
v2[:,:]=V(rho2,Vmax2)

On réutilise les mêmes fonctions fournies par Python pour trouver les minima et la moyenne.

In [14]:
print("La vitesse minimale à t=0 minute est:", min(v2[0,:])/3.6) 
print("La vitesse moyenne à t=3 minutes est:", sum(v2[int(3/60/dt),:])/nx/3.6) 
print("La vitesse minimale à t=6 minutes est:", min(v2[int(3/60/dt),:])/3.6) 

La vitesse minimale à t=0 minute est: 30.2222222222
La vitesse moyenne à t=3 minutes est: 33.872218191
La vitesse minimale à t=6 minutes est: 30.9864026806
